In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Initialisation des variables du problème

In [ ]:
lbd0 = np.array([0,0,0])
x0 = np.array([0,0])
A = np.array([[1, 1], [-1,1], [0,-1]])
b = np.array([1, 1, 1])
 
def surface(x,y):
    return ((x+y-1<=0) and (-x+y-1<=0) and (-y-1<=0))

def J(x, y):
    return 2*x**2-x*y+y**2-2*x+3*y-1

def dJ(x, y):
    return np.array([4*x-y-2, -x+2*y+3])

def proj_ortho (point):
    n = len(point)
    proj = np.zeros(n)
    for i in range(n):
        if point[i]>0:
            proj[i]=point[i]
    return proj


def Jt(x,y,T):
    S = 0
    X = np.array([x, y])
    ff = A@X-b
    for i in range (3):
        S=S+(T*np.log(-ff[i]))
    return J(x,y)-S

def dJt(x,y,T):
    S = np.zeros(2)
    X = np.array([x, y])
    ff = A@X-b
    for i in range (3):
        S=S+(T/ff[i])*A[i]
    return dJ(x,y)-S

# Tracé de la surface à minimiser

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x = np.linspace(-1, 1, 200)
y = np.linspace(-1, 1, 200)
for i in range (len(x)):
    for j in range (len(y)):
        if ((x[i]+y[j]-1<=0) and (-x[i]+y[j]-1<=0) and (-y[j]-1<=0))==False:
            x[i]=0
            y[j]=0

X, Y = np.meshgrid(x, y)
Z = J(X,Y)

fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1,
cmap='jet', edgecolor='none')
#ax.plot_surface(X, Y, S, rstride=1, cstride=1,cmap='jet', edgecolor='none')
ax.set_title("Surface", fontsize = 13)
ax.set_xlabel('x', fontsize = 11)
ax.set_ylabel('y', fontsize = 11)
ax.set_zlabel('Z', fontsize = 10)
plt.show()

# Algorithme d'Uzawa

D'après le tracé, il semble que x* = (0.25, -1).

On applique l'algorithme d'Uzawa et on trace pour chaque itération la norme de la différence entre la solution donnée par l'algorithme et la solution x*.

In [ ]:
def uzawa (tau, lbd, x, itermax, eps):
    solution = np.array([ 0.25, -1])
    i = 0
    liste = []
    while (i < itermax and np.linalg.norm(x-solution)>eps) :
        x = np.linalg.solve(np.array([[4, -1], [-1, 2]]), -A.T@lbd-np.array([-2, 3]))
        lbd = proj_ortho(lbd + tau*(A@x-b))
        i=i+1
        liste.append(np.linalg.norm(x-solution))
    plt.semilogy([i for i in range(1, len(liste)+1)], liste)
    plt.title('|xk −x∗| en fonctions des itérations')
    return x, lbd, i

print('x* = ', uzawa(0.1, lbd0, x0, 500, 10**-6)[0], ', lambda* = ', uzawa(0.1, lbd0, x0, 500, 10**-6)[1], ', nb itérations =', uzawa(0.1, lbd0, x0, 500, 10**-6)[2])

# Méthode des ponts intérieurs

On applique l'algorithme de line search sur Jt pour différentes valeurs de t, puis on trace pour chaque t la norme de la différence entre la solution donnée par l'algorithme et la solution x*

In [ ]:
def f(x,y):
    return Jt(x,y,T)

def df(x,y):
    return dJt(x,y,T)

def ponts_int(X):
    x = X[0]
    y = X[1]
    epsilon = 1e-6
    alpha, beta = 0.5, 0.75
    itermax = 2000
    iter_ls_max = 200
    ## initialisation 
    iter = 0
    t = 0.01
    fz = f(x,y)
    Z, F =[np.array([x, y])], [fz]
    flag = 'OK'

    ## Boucle d'optimisation
    while (iter < itermax):
        dx, dy = -df(x, y)
        d = np.hypot(dx, dy)
        if d < epsilon or flag == 'Not OK':
            break
        dd =d**2
        new_fz = f(x + t*dx, y + t*dy) 
        iter_ls = 0
        while (new_fz - fz + alpha*t*dd >=0):
            t=beta*t
            new_fz = f(x + t*dx, y + t*dy)
            iter_ls += 1
            if (iter_ls>=iter_ls_max):
                flag = 'Not OK'
                break
        #print("d = " + str(d) + ", f(z) - 1 =" + str(fz-1) + ", t= " +str(t))
        x, y, fz = x + t*dx, y + t*dy, new_fz
        Z.append(np.array([x, y]))
        F.append(fz)
        t = t/beta
        iter += 1 
    
    Z = np.array(Z)
    F = np.array(F)
    sol = Z[-1]
    return (iter, sol)

In [ ]:
solution = np.array([-0.5, 0])
liste_T = []
ecart = []
for i in range(1,7):
    T = 1/2**i
    liste_T.append(T)
    print("T = ", T, ", nbre d'itérations :", ponts_int(solution)[0], ", solution :", ponts_int(solution)[1])
    solution = ponts_int(solution)[1]
    ecart.append(np.linalg.norm(np.array([ 0.25, -1]-solution)))
                 
plt.plot(liste_T, ecart)
plt.title('|xT −x∗| en fonction de t')
plt.xlabel('T')
plt.ylabel('|xt −x∗|')
plt.show()

On a bien convergence de xt vers x* quand t tend vers 0.